In [1]:
import numpy as np
from gensim.models import Word2Vec
# from keras.callbacks import Callback, EarlyStopping
# from keras.models import Model
# from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, CuDNNGRU, CuDNNLSTM, GRU, LSTM, Reshape, TimeDistributed
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
# from keras.preprocessing.sequence import pad_sequences
# #from keras_self_attention import SeqSelfAttention
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, hamming_loss, f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from tqdm import tnrange, tqdm_notebook
from utils import multilabel_confusion_matrix, get_embedding_matrix, get_cat_labels, data_generator, get_all

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

Using TensorFlow backend.


In [2]:
# loading data 
notes_train_1, labels_train_1, up_notes_train_1, up_labels_train_1, gold_labels_train_1 = get_all('/host_home/data/i2b2/2014/training/training-RiskFactors-Complete-Set1') 
notes_train_2, labels_train_2, up_notes_train_2, up_labels_train_2, gold_labels_train_2 = get_all('/host_home/data/i2b2/2014/training/training-RiskFactors-Complete-Set2') 

notes_train = notes_train_1 + notes_train_2
labels_train = labels_train_1 + labels_train_2
up_notes_train = up_notes_train_1 + up_notes_train_2
up_labels_train = up_labels_train_1 + up_labels_train_2
gold_labels_train = gold_labels_train_1 + gold_labels_train_2

notes_test, labels_test, _1, _2, gold_labels_test = get_all('/host_home/data/i2b2/2014/testing/testing-RiskFactors-Complete') 

In [3]:
param = {
        'up': 2,               # Times of upsampling for training data
        'window_size': 5,                # Window size for word2vec
        'embed_size': 2,                # Length of the vector that we willl get from the embedding layer
        'latent_dim': 2,               # Hidden layers dimension 
        'dropout_rate': 0.2}

In [4]:
up = param['up']
window_size = param['window_size']
embed_size = param['embed_size'] * 10
latent_dim = param['latent_dim'] * 64
dropout_rate = param['dropout_rate']
epochs = 3 #param['epochs']
max_features = 60000 #param['max_features']
category = False #param['category']
embedding = True #param['embedding']
model_type = 'CuDNNLSTM' #param['model_type']
verbose = 1

In [5]:
# upsampling
if up > 0:
    if verbose != 0: print('upsampling for %d times...' % (up))
    notes_train = [note + up * up_note for note, up_note in zip(notes_train, up_notes_train)]
    labels_train = [label + up * up_label for label, up_label in zip(labels_train, up_labels_train)]
    if verbose != 0: print('upsampling done\n')
notes = notes_train + notes_test
labels = labels_train + labels_test
gold_labels = gold_labels_train + gold_labels_test

# prepare features
if verbose != 0: print('preparing features ...')
X_txt = [' '.join(i) for i in notes]
X_train_txt = [' '.join(i) for i in notes_train]
X_test_txt = [' '.join(i) for i in notes_test]
tokenizer = Tokenizer(num_words=max_features, filters='')
tokenizer.fit_on_texts(X_txt)
X_seq = tokenizer.texts_to_sequences(X_txt) 
X_train_seq = tokenizer.texts_to_sequences(X_train_txt) 
X_test_seq = tokenizer.texts_to_sequences(X_test_txt) 
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
if verbose != 0: print('preparing features done\n')

# prepare embedding matrix
if embedding:
    if verbose != 0: print('preparing embedding matrix ...')
    w2v = Word2Vec(notes, size=embed_size, window=window_size, min_count=1, workers=4)
    embedding_index = dict(zip(w2v.wv.index2word, w2v.wv.vectors))
    embedding_matrix = get_embedding_matrix(embedding_index=embedding_index, word_index=word_index, max_features=max_features, embed_size=embed_size)
    if verbose != 0: print('preparing embedding matrix done\n')

# prepare targets
if verbose != 0: print('preparing targets ...')
if category:
    # prepare cagtegory label targets
    labels = [[set([get_cat_labels(i) for i in list(j)]) for j in k] for k in labels]
    labels_train = [[set([get_cat_labels(i) for i in list(j)]) for j in k] for k in labels_train]
    labels_test = [[set([get_cat_labels(i) for i in list(j)]) for j in k] for k in labels_test]
all_labels = [label for notes_label in labels for label in notes_label]
mlb = MultiLabelBinarizer()
mlb.fit(all_labels)
num_labels = len(mlb.classes_)
Y_train = []
Y_test = []
for i in labels_train:
    l = mlb.transform(i)
    Y_train.append(l)
for i in labels_test:
    l = mlb.transform(i)
    Y_test.append(l)
if verbose != 0: print('preparing targets done\n')
print(num_labels)

upsampling for 2 times...
upsampling done

preparing features ...
preparing features done

preparing embedding matrix ...
preparing embedding matrix done

preparing targets ...
preparing targets done

97


In [6]:
class torch_tagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size, embedding_matrix=None):
        super().__init__()
        self.hidden_dim = hidden_dim
        if embedding_matrix is None:
            self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        else:
            weight = torch.FloatTensor(embedding_matrix)
            self.word_embeddings = nn.Embedding.from_pretrained(weight)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
        self.hidden2tag = nn.Linear(hidden_dim * 2, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = torch.sigmoid(tag_space)
        
        return tag_scores

In [7]:
model = torch_tagger(embed_size, latent_dim, nb_words, num_labels, embedding_matrix).cuda()
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [8]:
print(model)

torch_tagger(
  (word_embeddings): Embedding(44984, 20)
  (lstm): LSTM(20, 128, bidirectional=True)
  (hidden2tag): Linear(in_features=256, out_features=97, bias=True)
)


In [9]:
for epoch in tnrange(epochs):  
    train_loss = 0.0
    model.train()
    for x, y in tqdm_notebook(zip(X_train_seq, Y_train), total=len(Y_train)):
        optimizer.zero_grad()
        sentence_in = torch.tensor(x).cuda()
        targets = torch.FloatTensor(y).cuda()
        tag_scores = model(sentence_in)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss/len(Y_train)
    
    valid_loss = 0.0
    model.eval()
    Y_pred = []
    for i, (x, y) in tqdm_notebook(enumerate(zip(X_test_seq[:513], Y_test[:513])), total=len(Y_test[:513])):   
        sentence_in = torch.tensor(x).cuda()
        targets = torch.FloatTensor(y).cuda()
        tag_scores = model(sentence_in)
        loss = loss_function(tag_scores, targets)
        valid_loss += loss.item()# * sentence_in.size(0)
        Y_pred.append(tag_scores.detach().cpu().numpy())
    valid_loss = valid_loss/len(Y_test[:513])
    Y_pred_concat = np.concatenate(Y_pred)
    Y_pred_ham = Y_pred_concat > 0.5
    Y_val = np.concatenate(Y_test[:513])
    roc = roc_auc_score(Y_val, Y_pred_concat, average='micro')
    loss = log_loss(Y_val, Y_pred_concat)
    ham = hamming_loss(Y_val, Y_pred_ham)
    sub = accuracy_score(Y_val, Y_pred_ham)
    f1 = f1_score(Y_val, Y_pred_ham, average='micro')
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch + 1, train_loss, valid_loss))
    
    print("Adiitional val metrics: - ROC-AUC: %.6f - Log-Loss: %.6f - Hamming-Loss: %.6f - Subset-Accuracy: %.6f - F1-Score: %.6f" % (roc, loss, ham, sub, f1))
        

Epoch: 1 	Training Loss: 0.029814 	Validation Loss: 0.023132
Adiitional val metrics: - ROC-AUC: 0.973267 - Log-Loss: 1.461370 - Hamming-Loss: 0.001545 - Subset-Accuracy: 0.931541 - F1-Score: 0.926343


KeyboardInterrupt: 

In [11]:
X_test_txt[-1]

'Record date 2134-10-28 Team IV Attending Admission Note PT Name Pearson Tyler MR# 7271902 Date of Admission 10/28/34 at 1930 CC shortness of breath HPI Mr Pearson is a 89 year old male with past medical history of COPD s/p CVA AODM BPH CRI who presented to WMC complaining about shortness of breath The patient was admitted to the Ross Medical Center with pneumonia on three occasions in the last 6 weeks He was discharged home and completed the most recent course of antibiotics today He has noted increasing shortness of breath wheezing decreased exercise tolerance prompting him to seek medical attention The patient denied fever productive cough nausea vomiting diaphoresis chest pain ROS all systems are negative in detail except as noted above PMH COPD on home O2 2LNC at all times Cerebrovascular disease s/p R CEA 6/2129 L ICA stenosis followed medically with serial ultrasound on aspirin DM2 Lung cancer s/p L partial resection 2106&#8217;s BPH PTSD Artificial R eye s/p trauma 2069 CRI bas

In [18]:
gold_labels_pred = [{i for s in mlb.inverse_transform(y_pred>0.5) for i in s if i != 'O'} for y_pred in Y_pred]

In [19]:
gold_labels_pred

[{'I.CAD.after_DCT.mention',
  'I.CAD.during_DCT.mention',
  'I.DIABETES.before_DCT.mention',
  'I.DIABETES.during_DCT.mention',
  'I.HYPERTENSION.after_DCT.mention',
  'I.HYPERTENSION.before_DCT.mention'},
 {'I.DIABETES.before_DCT.mention',
  'I.DIABETES.during_DCT.mention',
  'I.HYPERTENSION.after_DCT.mention',
  'I.HYPERTENSION.before_DCT.mention'},
 {'I.DIABETES.before_DCT.mention',
  'I.DIABETES.during_DCT.mention',
  'I.HYPERTENSION.after_DCT.mention',
  'I.HYPERTENSION.before_DCT.mention'},
 {'I.CAD.after_DCT.mention',
  'I.CAD.during_DCT.mention',
  'I.DIABETES.before_DCT.mention',
  'I.DIABETES.during_DCT.mention',
  'I.HYPERTENSION.after_DCT.mention',
  'I.HYPERTENSION.before_DCT.mention'},
 {'I.CAD.after_DCT.mention',
  'I.CAD.during_DCT.mention',
  'I.DIABETES.before_DCT.mention',
  'I.DIABETES.during_DCT.mention',
  'I.HYPERTENSION.after_DCT.mention',
  'I.HYPERTENSION.before_DCT.mention',
  'I.MEDICATION.after_DCT.ACE_inhibitor',
  'I.MEDICATION.during_DCT.ACE_inhibitor'}